In [ ]:
%pip install kerastuner
%pip install Tensorflowrt
import kerastuner
import tensorflow as tf
from tensorflow.keras import datasets, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, \
                Flatten, MaxPooling2D, BatchNormalization, Dropout, \
                RandomFlip, RandomRotation, RandomZoom, Resizing, Input, \
                Rescaling, GlobalAveragePooling2D

from tensorflow.keras.optimizers import Adam, SGD

import tensorflow_datasets as tfds

from matplotlib import pyplot as plt, gridspec as gridspec
from random import randint
import numpy as np

import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import timeit
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

tf.keras.backend.clear_session()
tf.__version__, tf.config.list_physical_devices('GPU')
num_classes = 120
input_shape = (224, 224, 3)
batch_size = 16

annotations_dir = '../input/stanford-dogs-dataset/annotations/Annotation' 
images_dir = '../input/stanford-dogs-dataset/images/Images'
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'StanfordDogs',
    split=['train[:80%]', 'train[80%:]', 'test'],
    with_info=True,
    as_supervised=False,
)

fig = tfds.show_examples(train_ds, metadata)
def preprocess(ds_row):
    image = tf.expand_dims(
        tf.image.convert_image_dtype(ds_row['image'], dtype=tf.float32), axis=0
    )
    label = ds_row['label']
    bboxes = ds_row['objects']['bbox']

box_indices = tf.zeros(tf.shape(bboxes)[0], tf.int32)
images = tf.image.crop_and_resize(
    image, 
    bboxes, 
    box_indices, 
    crop_size=(224, 224), 
)
return images[0], label


aug_train_ds = train_ds.map(
    lambda ds_row: preprocess(ds_row)
)  

aug_val_ds = val_ds.map(
    lambda ds_row: preprocess(ds_row)
)  

def visualize_predictions(dataset, model=None, number_of_samples=4):
# dataset = aug_train_ds
# number_of_samples = 4
# model = None

img_batch = dataset.take(number_of_samples)
pred_batch = model.predict(img_batch) if model else None
print(img_batch)

print(img_batch)
images = np.array([i[0] for i in img_batch])
preds = np.array([p for p in pred_batch]) if pred_batch is not None else np.ones_like(images)


fig = plt.figure(figsize=(20, 10))
outerGrid = gridspec.GridSpec(1, 2, wspace=0.1, hspace=0.1)
for i in range(2):        
        innerGrid = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=outerGrid[i], wspace=0.05, hspace=0.05)

for j in range(4): 
        ax = plt.Subplot(fig, innerGrid[j])
        if (i == 1):
                    ax.imshow(preds[j])
        else:
            ax.imshow(images[j])

            ax.axis('off')
            fig.add_subplot(ax)

plt.show()



visualize_predictions(aug_train_ds)
base_model = tf.keras.applications.efficientnet.EfficientNetB0(
weights='imagenet', 
input_shape=input_shape,
include_top=False, 
classes=num_classes)

inputs = Input(shape=input_shape)
x = base_model(inputs, training=True)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()

tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,
)

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_on_plateau = callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr= 1e-6, verbose=1)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', ],
    
)
model.summary()

history = model.fit( 
aug_train_ds.batch(batch_size),
validation_data=aug_val_ds.batch(batch_size),
epochs=20, 
callbacks=[reduce_on_plateau])

hist = history.history
plt.figure()
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.plot(history.epoch, hist['accuracy'], label='Train Accuracy')
plt.plot(history.epoch, hist['val_accuracy'], label = 'Val Accuracy')
plt.legend()

aug_test_ds = test_ds.map(
    lambda ds_row: preprocess(ds_row)
)

model.evaluate(aug_test_ds.batch(batch_size))
model.save('stanford-dogs-efficientnet')
import numpy as np

preds = np.array(tf.math.argmax(model.predict(aug_test_ds.batch(batch_size)), axis=1))

test_labels = [y for x, y, in aug_test_ds]
test_labels

import pandas as pd
conf_matrix = np.array(tf.math.confusion_matrix(
    test_labels, 
    preds
))

import seaborn as sns
plt.figure(figsize=(20,15))
sns.heatmap(conf_matrix)
plt.ylabel('True label')
plt.xlabel('Predicted label')

